In [1]:
import numpy as np
import scipy.linalg as la
import time

In [2]:
def least_squares(X, Y):
    """Good Luck!"""
    return la.solve(X.T.dot(X), X.T.dot(Y))

def qr_least_squares(X,Y):
    """You can do it!"""
    n,d = X.shape
    Q,R = la.qr(X)
    return la.solve_triangular(R[:d],Q.T.dot(Y)[:d])

## 10.10. 
Code up the method of the previous problem for finding the estimator $\beta$ using the SVD.

(i) Let $n = 10^3$. Generate artificial data with three independent variables $d = 3$, by setting $X = np.random.random((n, 3))$ and let $\beta= [1, 2, 3]$. Set $e = np.random.randn(n)$, and $y = np.dot(X, beta) + e$. 

(ii) Run your solver on the generated data $X$ and $Y$ and compare the result to the known value of $\beta$. Compare the results (both for speed and accuracy) with the two methods you coded up on the last assignment (using a naïve solver and $QR$ decompositon).

(iii) Let $n = 10^3$. Generate artificial data with $d = 6$ and $r = 3$, by setting $Z = np.random.random((n, 3))$ and $X = np.hstack([Z,Z])$. Let $\beta =  [1, 2, 3, 1, 2, 3]$. Set $e = np.random.randn(n)$, and $y = np.dot(X, beta) + e$ and run your solver on this dataset. Compare to the known value of $\beta$. How big are the residuals $Y - Yˆ$ ?

In [3]:
def svd_ols(X,Y):
    '''
    Calculates the basic OLS solution using
    the singular value decomposition
    '''
    n,d = X.shape
    U, s, Vh = la.svd(X)
    r = np.argmin(s)
    sig1inv = np.diag(1./s[:r])
    V1 = Vh.T[:d,:r]
    A = U.T.dot(Y)[:r]
    beta = V1.dot(sig1inv.dot(A))
    return beta

In [4]:
# part i
n = 10**3
d = 3
X = np.random.random((n,d))
beta = np.array([1,2,3])
e = np.random.randn(n)
y = np.dot(X,beta) + e

In [9]:
# part ii
start = time.clock()
bhat_svd = svd_ols(X,y)
svd_time = time.clock() - start
svd_error = la.norm(bhat_svd - beta)

start = time.clock()
bhat_qr = qr_least_squares(X,y)
qr_time = time.clock() - start
qr_error = la.norm(bhat_qr - beta)

start = time.clock()
bhat_naive = least_squares(X,y)
naive_time = time.clock() - start
naive_error = la.norm(bhat_naive - beta)

print("\tTime:\t\t\tAccuracy:\nSVD:\t{}\t{}\nQR: \t{}\t\t{}\nNaive:\t{}\t{}".format(
    svd_time, svd_error, qr_time, qr_error, naive_time, naive_error))

	Time:			Accuracy:
SVD:	0.00819732324432	0.742563492177
QR: 	0.020869307648		0.189311672004
Naive:	0.000317948326629	0.189311672004


The accuracy is defined to be $||  \hat{\beta}_{method} - \beta  ||$

In [6]:
# part iii
n2 = 10**3
d2 = 6
r2 = 3
Z = np.random.random((n2,3))
X2 = np.hstack([Z,Z])
beta2 = np.array([1,2,3,1,2,3])
e2 = np.random.randn(n)
y2 = np.dot(X2, beta2) + e

In [7]:
bhat2 = svd_ols(X2, y2)
ehat2 = y2 - X2.dot(bhat2)
print_statement = "Error for beta hat:\t\t{}\nSum of Squared Residuals:\t{}\nR^2:\t\t\t\t{}"
print(print_statement.format(la.norm(bhat2 - beta2), 
                       np.dot(ehat2,ehat2), 
                       1 - np.dot(ehat2,ehat2)/np.dot(y2-y2.mean(), y2-y2.mean())))

Error for beta hat:		1.05046324864e+15
Sum of Squared Residuals:	972.324144145
R^2:				0.83485073638
